# [7、深入剖析PyTorch nn.Module源码](https://www.bilibili.com/video/BV1ch411b7yP?spm_id_from=333.788.player.switch&vd_source=cdd897fffb54b70b076681c3c4e4d45d)

其感觉也不深入


In [ ]:
import torch
import torch.nn as nn
from torch.nn import Module
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset, DataLoader

## nn.Module

基本类，自定义模型都需要继承此类

- add_module：增加模块，可以通过 . 来访问
- apply：使用一个函数应用到所有子模块，一般由于初始化参数
- bfloat16：浮点类型都转换为 bf16 低精度
- buffers：返回模块buffer的迭代器，神经网络中参数用来随机梯度下降，buffer不参与。例如BatchNorm
- eval：设置为evaluate模式，其实就是 train(False)
- load_state_dict：从磁盘保存模型的参数 （torch.save的）
- named_parameters：返回所有模型参数
- requires_grad_：是否需要梯度更新（在GAN、微调中用到）

## apply

In [ ]:
# apply：使用一个函数应用到所有子模块，一般由于初始化参数
@torch.no_grad()
def init_weights(m):
    print(m)
    if type(m) == nn.Linear:
        m.weight.fill_(1.0)
        print(m.weight)


net = nn.Sequential(nn.Linear(2, 2), nn.Linear(2, 2))
net.apply(init_weights)

Linear(in_features=2, out_features=2, bias=True)
Parameter containing:
tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
Linear(in_features=2, out_features=2, bias=True)
Parameter containing:
tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): Linear(in_features=2, out_features=2, bias=True)
)


Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): Linear(in_features=2, out_features=2, bias=True)
)

## `register_buffer`

有些模型的数据不参与梯度更新，但是是模型的内容，因此需要设置为buffer不参与梯度计算。

- `self._buffers[name] = tensor`

例如 BatchNorm 中均值是统计信息。

## `register_parameter`

注册参数，会被更新。
- `self._parameters[name] = param`

param类型的变量，如果用在 Module 中，会被自动加入_parameters表中

## `get_submodule`

```txt
A(
    (net_b): Module(
        (net_c): Module(
            (conv): Conv2d(16, 33, kernel_size=(3, 3), stride=(2, 2))
        )
        (linear): Linear(in_features=100, out_features=200, bias=True)
    )
)
```

存在嵌套结构，使用`.`：`get_submodule("net_b.net_c.conv")`